In [ ]:
import seaborn as sns

import sklearn
from sklearn.cluster.bicluster import SpectralBiclustering
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.metrics import consensus_score

import scipy.linalg
from functools import partial
%pylab
%matplotlib inline
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import scipy.sparse as sparse
from scipy import stats
from __future__ import print_function
import gc
from tqdm import tqdm_notebook as tqdm
from __future__ import division
import pandas as pd
import seaborn as sb
import os
from scipy.spatial import distance
from scipy.cluster import hierarchy
from pandas import DataFrame

import os, sys, datetime
LF_CODE_PATH = os.path.expanduser('~/projects/LFAnalyze/code')
FT_CODE_PATH = os.path.expanduser('~/projects/fishTrax/code/analysis/')
FD_CODE_PATH = os.path.expanduser('~/projects/fish_despair_notebooks/src/')
sys.path.insert(0,LF_CODE_PATH)
sys.path.insert(0,FT_CODE_PATH)
sys.path.insert(0,FD_CODE_PATH)

import passivity_2p_imaging_utils as p2putils
import visualization_utils as vizutil
import matplotlib.cm as cm
from matplotlib.colors import Normalize
reload(p2putils)
tmp_dir = '/tmp/'
all_data = p2putils.get_all_datasets(tmp_dir=tmp_dir)

fishIdx = [("e", 2),  ("e", 5), ("c", 1),  ("c", 6),  ("enp", 1), ("enp", 5)]
indicator = 0

f = all_data[fishIdx[indicator][0]][fishIdx[indicator][1]]


import tensorflow as tf
# tf.enable_eager_execution()


# df, sig = f.get_cnmf_roi_table_and_signals()
M = f.get_signals_raw(z=None)

# data = np.load("../cnmf_f01555.npz")
# cnmf = data['cnmf'].astype(np.float32)
# raw = data['raw'].astype(np.float32)
# del data

def moving_average(a, n=3):
    # a is a signal
    ret = np.cumsum(a,0) # sum over time
    ret[n:] = ret[n:] - ret[:-n] # diff of n samples back
    rm = ret[n - 1:] / n
    pad_start = np.full((n-1,rm.shape[1]), rm[0])
    return np.vstack([pad_start, rm])

def ewma(data,span):
    "exponential weighted moving average."
    df = DataFrame(data)
    return df.ewm(span).mean().values

def df_f(x,ma_window=6,span=6):
    u = moving_average(x,ma_window)
    return ewma((x - u)/u, span)

In [ ]:
# neurons = sig.T
neurons = M.T

# neuron_ids = np.sort(np.argsort(neurons.std(0))[-5000:])
df = f.get_roi_table()
# df = df.iloc[neuron_ids]
# df.reset_index(drop=True, inplace=True)
# neurons = neurons[:,neuron_ids]

from scipy import signal

neurons = signal.decimate(neurons,4, axis=0)

neurons = df_f(neurons).astype(np.float32)
neurons = (neurons - neurons.mean(0))/(neurons.std(0)+1e-8)

In [ ]:
from sklearn.cross_decomposition import CCA
from sklearn.decomposition import PCA

In [ ]:
ortho_regions = [u'in_l_cerebellum', u'in_r_cerebellum', 
    u'in_l_vthal', u'in_r_vthal', u'in_l_tectum', u'in_r_tectum', 
    u'in_l_raphe', u'in_r_raphe', u'in_l_hind', u'in_r_hind', 
    u'in_l_dthal', u'in_r_dthal', u'in_l_LHb', u'in_r_LHb', 
    u'in_l_tel', u'in_r_tel', u'in_l_MHb',  u'in_r_MHb']

regions = []
for r in ortho_regions:
    if r in df.columns:
        regions.append(r)

In [ ]:
region_pca = {}
for r in regions:
    X = neurons[:,df[r]]
    pca = PCA(n_components=1)
    pca.fit(X)
    transform = pca.fit_transform(X)
    region_pca[r] = {"components": pca.components_, "explained_var": pca.explained_variance_ratio_, "transform": transform}
neurons_pca_transform = np.concatenate([region_pca[r]['transform'] for r in regions],1)

In [ ]:
def construct_pairwise_batches(X, lag=1, startIdx=None, endIdx=None, bias=True):
    """constructs batch x nT-lag x nBeta. i*ncol+j index is granger model of influence from i->j.
    
    Use startIdx and endIdx to choose a certain number of rows,
    eg startIdx=0, endIdx=5, X.shape[1]=100 would be a batch of 100^2=10,000.
    
    WARNING: if endIdx>N, will fill with 1s so batch stays same size."""
    
    nT, N = X.shape
    if endIdx:
        bz = (endIdx - startIdx) * N
    else:
        startIdx=0
        endIdx=N
        bz = N**2
#     print([bz, nT-lag, 2*lag+bias])
    granger = np.ones([bz, nT-lag, 2*lag+bias])
    null = np.ones([bz, nT-lag, lag+bias])
    newY = np.ones([bz, nT-lag, 1])
    Y = X[lag:]
    # time x neuron x lag
    l = 0
    X_lag = np.concatenate([X[l:-(lag-l)][:,:,None] for l in reversed(range(lag))], 2)
    biasTerm = np.ones([nT-lag,1])
    for i, n1 in enumerate(range(startIdx,min(N,endIdx))):
        for j, n2 in enumerate(range(N)):
            if bias:
#                 print([x.shape for x in [X_lag[:,n1], X_lag[:,n2], biasTerm]])
                grow = np.concatenate([X_lag[:,n1], X_lag[:,n2], biasTerm], 1)
                nrow = np.concatenate([X_lag[:,n2], biasTerm], 1)
            else:
                grow = np.concatenate([X_lag[:,n1], X_lag[:,n2]], 1)
                nrow = np.concatenate([X_lag[:,n2]], 1)
#             print(i,j)
            granger[i*N+j] = grow
            null[i*N+j] = nrow
            newY[i*N+j] = Y[:,[n2]]
    return newY, granger, null

def f_value(resA, resB, n_params_A, n_params_B, n):
    return ( (resA-resB)/(n_params_B-n_params_A) ) / (resB/(n-n_params_B))

class Granger():
    def __init__(self, x_granger_shape, x_null_shape, y_shape, sess, l2=0.0):
        self.x_granger = tf.placeholder(dtype=tf.float32, shape=x_granger_shape)
        self.x_null = tf.placeholder(dtype=tf.float32, shape=x_null_shape)
        self.y = tf.placeholder(dtype=tf.float32, shape=y_shape)
        self.n_params_granger = x_granger_shape[-1]
        self.n_params_null = x_null_shape[-1]
        self.l2 = l2
        self.sess = sess
        
    def granger(self, X, Y):
        gbeta = tf.matrix_solve_ls(self.x_granger, self.y, l2_regularizer=self.l2, fast=False)
        g_residuals = tf.reduce_sum(((tf.matmul(self.x_granger,gbeta) - self.y)**2)[:,:,0], 1)
        gbeta, g_residuals = self.sess.run([gbeta, g_residuals],
                                      feed_dict={self.x_granger: X, self.y: Y})
        return gbeta, g_residuals
    
    def null(self, X, Y):
        gbeta = tf.matrix_solve_ls(self.x_null, self.y, l2_regularizer=self.l2, fast=False)
        g_residuals = tf.reduce_sum(((tf.matmul(self.x_null,gbeta) - self.y)**2)[:,:,0], 1)
        gbeta, g_residuals = self.sess.run([gbeta, g_residuals],
                                      feed_dict={self.x_null: X, self.y: Y})
        return gbeta, g_residuals

def pairwise_granger_f_val(neurons, lag=1, bz=10):
    "all_gbeta is granger then autocorr then bias."
    nT, N = neurons.shape
    Y, x_granger, x_null = construct_pairwise_batches(neurons,lag,0,bz)
    fvals = np.zeros([N,N])
    all_gbeta = np.zeros([N,N,2*lag+1])
    all_nbeta = np.zeros([N,N,lag+1])
    with tf.Session() as sess:
        granger = Granger(x_granger.shape, x_null.shape, Y.shape, sess=sess, l2=0.0)
        for start in tqdm(range(0,N,bz)):
            end = start+bz
            Y, x_granger, x_null = construct_pairwise_batches(neurons,1,start,end)
            gbeta, g_residuals = granger.granger(x_granger, Y)
            nbeta, n_residuals = granger.null(x_null, Y)
            batch_fvals = f_value(g_residuals, n_residuals, granger.n_params_granger, granger.n_params_null, N)
            # account for N % batch != 0
            true_end = min(start+bz,N)
            batch_fvals = batch_fvals.reshape(-1,N)
            batch_gbeta = gbeta.reshape(-1,N,2*lag+1)
            batch_nbeta = nbeta.reshape(-1,N,lag+1)
            if start+bz>N:
                batch_true_end = N-start
                batch_fvals = batch_fvals[:batch_true_end]
                batch_gbeta = batch_gbeta[:batch_true_end]
                batch_nbeta = batch_nbeta[:batch_true_end]
            fvals[start:true_end] = batch_fvals
            all_gbeta[start:true_end] = batch_gbeta
            all_nbeta[start:true_end] = batch_nbeta
    return fvals, all_gbeta, all_nbeta

In [ ]:
fvals, gbeta, nbeta = pairwise_granger_f_val(neurons_pca_transform, 1, 10)

In [ ]:
reg = list(map(lambda x: x[3:],regions))
plt.figure(figsize=(10,10))
plt.imshow(fvals,vmin=0,vmax=3.5)
plt.colorbar()
plt.xticks(np.arange(len(reg)), reg)
plt.yticks(np.arange(len(reg)), reg)
plt.title("{} F-value for granger causality on first PCA".format(f.fishid))

# Archive
mostly CCA...

In [ ]:
X = neurons_pca_transform[:-1]
Y = neurons_pca_transform[1:]
n_components = 10
cca = CCA(n_components=n_components)
cca.fit(X,Y)
CCA(copy=True, max_iter=500, n_components=1, scale=True, tol=1e-06)
# cca.x_weights_
# cca.y_weights_

In [ ]:
plt.subplot(1,3,1)
plt.imshow(cca.x_weights_)
plt.subplot(1,3,2)
plt.imshow(cca.y_weights_)
plt.subplot(1,3,3)
plt.imshow(cca.y_weights_-cca.x_weights_) 

In [ ]:
reg = list(map(lambda x: x[3:],regions))
plt.figure(figsize=(10,10))
plt.imshow(fvals)
plt.colorbar()
plt.xticks(np.arange(len(reg)), reg)
plt.yticks(np.arange(len(reg)), reg)
plt.title("{} F-value for granger causality on first PCA".format(f.fishid))

In [ ]:
vmax = np.percentile(cca.x_weights_,99)
vmin = np.percentile(cca.x_weights_,1)
# vmean = np.mean(cca.x_weights_)
cmap_ = cm.bwr
# mpl.colors.Normalize(vmin=-1.,vmax=1.)
norm_ = Normalize(vmin=vmin, vmax=vmax)
m = cm.ScalarMappable(norm=norm_, cmap=cmap_)
cmap = lambda x: m.to_rgba(x)[:3]
p = np.array([map(cmap, cca.x_weights_[:100,i]) for i in range(n_components)])
plt.imshow(p, vmin=0, vmax=1)
# plt.colorbar()

In [ ]:
Z = [0,2,4,6,8,10]
# Z = [4]
nZ = len(Z)
back_img = []
for z in Z:
    back_img.append(np.power(f.get_tif_rasl_as_vol(z,range(1,200)).mean(axis=2),.4))
    

In [ ]:
def overlay_color_coords(img, roi_coords, colors, alpha):
    """
    Create an new image by overlaying per ROI coloring (each a set of coordinates) on backgroung image.
    Note, this function currently rescales the background img to use the entire grayscale range (change this).
    
    img - a grayscale (single channel) image.
    roi_coords - a list of N  (Mx2) arrays specifing sets of coords for each roi (N rois)
    colors - a list of N rgb colors, e.g red is 1,0,0.
    """
    import numpy as np

    over_img = (img - img.min()) / float(img.max() - img.min())
    over_img = np.dstack((over_img, over_img, over_img))

    if len(roi_coords) == 0:
        return over_img
    if (len(np.array(colors).shape)==1) and (np.array(colors).shape[0] == 3):
        print('multi')
        colors = np.array([colors]*len(roi_coords))
    assert colors.shape[0] == len(roi_coords)

    for coords, color in zip(roi_coords, colors):
        over_img[coords[:,0],coords[:,1],:]*=(1-alpha)
        over_img[coords[:,0],coords[:,1],:]+=color*alpha
        #for x,y in coords:
        #    over_img[x,y,:] = over_img[x,y,:]*(1-alpha)+color*alpha
        #skimage.draw.set_color(img = over_img, 
        #                       coords = (coords[:,0],coords[:,1]),
        #                       color = color)   
    return over_img

# nrows = len(row_clust_to_plot)
# nZ = 1
nrows = n_components
plt.subplots(nrows,nZ, figsize=[8*nZ,4*nrows])
for ic, clust in enumerate(range(1, n_components+1)):
# for ic,clust in enumerate(row_clust_to_plot):
    for iz, z in enumerate(Z):
        idx = df.z==z
        #Select rois in raphe in this slices, and get their coordinates.
        coords = df[idx].coords
        my_colors = np.array(map(cmap, cca.x_weights_[idx,ic]))
#         poly_coords = df[(row_clusters==clust) & (df.z==z)].poly
#         poly_coords = df[(row_clusters==clust)].poly
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nrows,nZ,ic*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        # hack iz hardcode
        img = overlay_color_coords(back_img[iz], coords, my_colors, alpha=0.5)
        plt.imshow(img,interpolation='nearest')
        plt.title("t-1 cluster {}, z={}".format(clust,z),fontsize=18)

In [ ]:
# nrows = len(row_clust_to_plot)
# nZ = 1
nrows = n_components
plt.subplots(nrows,nZ, figsize=[8*nZ,4*nrows])
for ic, clust in enumerate(range(1, n_components+1)):
# for ic,clust in enumerate(row_clust_to_plot):
    for iz, z in enumerate(Z):
        idx = df.z==z
        #Select rois in raphe in this slices, and get their coordinates.
        coords = df[idx].coords
        my_colors = np.array(map(cmap, cca.y_weights_[idx,ic]))
#         poly_coords = df[(row_clusters==clust) & (df.z==z)].poly
#         poly_coords = df[(row_clusters==clust)].poly
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nrows,nZ,ic*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        # hack iz hardcode
        img = overlay_color_coords(back_img[iz], coords, my_colors, alpha=0.5)
        plt.imshow(img,interpolation='nearest')
        plt.title("t-1 cluster {}, z={}".format(clust,z),fontsize=18)

In [ ]:
# nrows = len(col_clust_to_plot)
plt.subplots(nrows,nZ, figsize=[8*nZ,4*nrows])
for ic,clust in enumerate(range(1,g_nclust+1)):
# for ic,clust in enumerate(col_clust_to_plot):
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
        coords = df[(col_clusters==clust) & (df.z==z)].coords
#         poly_coords = df[(row_clusters==clust) & (df.z==z)].poly
#         coords = df[(col_clusters==clust)].coords
#         poly_coords = df[(col_clusters==clust)].poly
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nrows,nZ,ic*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        img = vizutil.overlay_coords(back_img[iz], coords, list(cm_cycle(clust)[:3]), alpha=1)
        plt.imshow(img,interpolation='nearest')
        plt.title("Col cluster {}, z={}".format(clust,z),fontsize=18)